pip install pulp

In [1]:
from pulp import *   # PuLP - Python Library for Linear Programming
import pandas as pd
import numpy as np
import math

In [2]:
n_supply = 100   #number of supply points
n_demand = 4096  #number of demand points

#Importing csv files 
demand_set = pd.read_csv('Prediction.csv') 
supply_set = pd.read_csv('exisiting_EV_infrastructure_2018.csv')
distij=pd.read_csv('distijtrans.csv')

In [3]:
#storing the distance matrix in 2D array
dist_matrix=distij.to_numpy()
df1=pd.DataFrame(dist_matrix)
df1

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,53.142156,52.208836,51.278030,50.349877,49.424527,48.502141,47.582890,46.666961,45.754552,44.845878,...,44.300780,44.409828,44.541067,44.694300,44.869301,45.065818,45.283571,45.522253,45.781539,46.061080
1,68.387858,67.854449,67.331667,66.819761,66.318983,65.829588,65.351830,64.885968,64.432258,63.990961,...,17.987223,18.943792,19.904632,20.869151,21.836863,22.807360,23.780302,24.755401,25.732411,26.711122
2,73.397670,72.772244,72.155256,71.546925,70.947474,70.357129,69.776122,69.204687,68.643064,68.091495,...,9.817786,10.628756,11.469612,12.334242,13.217983,14.117245,15.029243,15.951792,16.883164,17.821974
3,62.558091,62.084277,61.623049,61.174692,60.739490,60.317729,59.909692,59.515661,59.135917,58.770736,...,25.309363,26.258458,27.211199,28.167217,29.126188,30.087831,31.051898,32.018168,32.986449,33.956569
4,61.656645,60.831721,60.012122,59.198068,58.389791,57.587534,56.791553,56.002116,55.219502,54.444006,...,28.539067,28.660684,28.816510,29.005994,29.228481,29.483225,29.769397,30.086100,30.432382,30.807244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,49.423023,48.514871,47.610402,46.709827,45.813377,44.921299,44.033859,43.151342,42.274057,41.402337,...,43.449039,43.667798,43.908247,44.170032,44.452777,44.756083,45.079536,45.422705,45.785147,46.166409
96,44.393460,43.728052,43.075582,42.436646,41.811866,41.201885,40.607371,40.029012,39.467518,38.923620,...,38.619456,39.253351,39.902240,40.565404,41.242153,41.931830,42.633808,43.347489,44.072304,44.807714
97,41.812426,40.961461,40.117373,39.280605,38.451635,37.630979,36.819192,36.016874,35.224672,34.443284,...,45.132448,45.546682,45.978936,46.428707,46.895492,47.378788,47.878093,48.392914,48.922760,49.467149
98,56.712784,55.968438,55.232167,54.504297,53.785171,53.075143,52.374583,51.683877,51.003424,50.333641,...,27.887728,28.316983,28.774602,29.259253,29.769616,30.304393,30.862314,31.442146,32.042702,32.662837


In [4]:
#Demand values of 2018
ev_demands=np.array(demand_set['2019'])
ev_demands

array([15.08853972, 13.45382863, 17.50160494, ...,  3.78593664,
        7.28377172,  7.93874021])

#Smax value of each supply point
supply_max=np.array(supply_set['smax'])
supply_max

In [5]:
#Initialize the linear optimization model
model = LpProblem("Supply-Demand-Problem", LpMinimize)
model

Supply-Demand-Problem:
MINIMIZE
None
VARIABLES

In [6]:
#Creating unique index number for each value in DSij matrix
index_numbers = [str(i) for i in range(100*4096)]
index_numbers.sort()
print("Variable Indices:", index_numbers)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
#Creating unique variables using index numbers
DV_variables = LpVariable.matrix("X", index_numbers, cat = "Continuous", lowBound= 0 )
dsij_mat = np.array(DV_variables).reshape(n_supply,n_demand)
print("Decision Variable/Allocation Matrix: ")
print(dsij_mat)

Decision Variable/Allocation Matrix: 
[[X_0 X_1 X_10 ... X_103680 X_103681 X_103682]
 [X_103683 X_103684 X_103685 ... X_107367 X_107368 X_107369]
 [X_10737 X_107370 X_107371 ... X_111052 X_111053 X_111054]
 ...
 [X_8894 X_88940 X_88941 ... X_92622 X_92623 X_92624]
 [X_92625 X_92626 X_92627 ... X_96309 X_9631 X_96310]
 [X_96311 X_96312 X_96313 ... X_99997 X_99998 X_99999]]


In [8]:
df=pd.DataFrame(dsij_mat)
df

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,X_0,X_1,X_10,X_100,X_1000,X_10000,X_100000,X_100001,X_100002,X_100003,...,X_103674,X_103675,X_103676,X_103677,X_103678,X_103679,X_10368,X_103680,X_103681,X_103682
1,X_103683,X_103684,X_103685,X_103686,X_103687,X_103688,X_103689,X_10369,X_103690,X_103691,...,X_107360,X_107361,X_107362,X_107363,X_107364,X_107365,X_107366,X_107367,X_107368,X_107369
2,X_10737,X_107370,X_107371,X_107372,X_107373,X_107374,X_107375,X_107376,X_107377,X_107378,...,X_111046,X_111047,X_111048,X_111049,X_11105,X_111050,X_111051,X_111052,X_111053,X_111054
3,X_111055,X_111056,X_111057,X_111058,X_111059,X_11106,X_111060,X_111061,X_111062,X_111063,...,X_114733,X_114734,X_114735,X_114736,X_114737,X_114738,X_114739,X_11474,X_114740,X_114741
4,X_114742,X_114743,X_114744,X_114745,X_114746,X_114747,X_114748,X_114749,X_11475,X_114750,...,X_11842,X_118420,X_118421,X_118422,X_118423,X_118424,X_118425,X_118426,X_118427,X_118428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,X_81566,X_81567,X_81568,X_81569,X_8157,X_81570,X_81571,X_81572,X_81573,X_81574,...,X_85243,X_85244,X_85245,X_85246,X_85247,X_85248,X_85249,X_8525,X_85250,X_85251
96,X_85252,X_85253,X_85254,X_85255,X_85256,X_85257,X_85258,X_85259,X_8526,X_85260,...,X_88930,X_88931,X_88932,X_88933,X_88934,X_88935,X_88936,X_88937,X_88938,X_88939
97,X_8894,X_88940,X_88941,X_88942,X_88943,X_88944,X_88945,X_88946,X_88947,X_88948,...,X_92616,X_92617,X_92618,X_92619,X_9262,X_92620,X_92621,X_92622,X_92623,X_92624
98,X_92625,X_92626,X_92627,X_92628,X_92629,X_9263,X_92630,X_92631,X_92632,X_92633,...,X_96302,X_96303,X_96304,X_96305,X_96306,X_96307,X_96308,X_96309,X_9631,X_96310


In [9]:
#Creating unique index number for each value in DSij matrix
index_numbers_y = [str(i) for i in range(100)]
index_numbers_y.sort()
print("Variable Indices:", index_numbers_y)

Variable Indices: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [10]:
#Creating unique variables using index numbers
Smax=[]
rem_slot=supply_set['total_parking_slots']-(supply_set['existing_num_SCS']+supply_set['existing_num_FCS'])
rem_slot.to_numpy()
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,8].values)
    ub=rem_slot[i]*400
    temp=LpVariable('y_{}'.format(index_numbers_y[i]),cat='Integer',lowBound=lb,upBound=lb+ub)
    Smax.append(temp)
Smax=np.array(Smax)
Smax

[y_0,
 y_1,
 y_10,
 y_11,
 y_12,
 y_13,
 y_14,
 y_15,
 y_16,
 y_17,
 y_18,
 y_19,
 y_2,
 y_20,
 y_21,
 y_22,
 y_23,
 y_24,
 y_25,
 y_26,
 y_27,
 y_28,
 y_29,
 y_3,
 y_30,
 y_31,
 y_32,
 y_33,
 y_34,
 y_35,
 y_36,
 y_37,
 y_38,
 y_39,
 y_4,
 y_40,
 y_41,
 y_42,
 y_43,
 y_44,
 y_45,
 y_46,
 y_47,
 y_48,
 y_49,
 y_5,
 y_50,
 y_51,
 y_52,
 y_53,
 y_54,
 y_55,
 y_56,
 y_57,
 y_58,
 y_59,
 y_6,
 y_60,
 y_61,
 y_62,
 y_63,
 y_64,
 y_65,
 y_66,
 y_67,
 y_68,
 y_69,
 y_7,
 y_70,
 y_71,
 y_72,
 y_73,
 y_74,
 y_75,
 y_76,
 y_77,
 y_78,
 y_79,
 y_8,
 y_80,
 y_81,
 y_82,
 y_83,
 y_84,
 y_85,
 y_86,
 y_87,
 y_88,
 y_89,
 y_9,
 y_90,
 y_91,
 y_92,
 y_93,
 y_94,
 y_95,
 y_96,
 y_97,
 y_98,
 y_99]

In [11]:
#Defining the Customer Dissatisfaction cost

cost_eq = lpSum(dsij_mat*dist_matrix)

In [12]:
#Adding the cost equation to the model
model +=  cost_eq
#print(model)

In [13]:
#Supply Constraint
#Column summation must be less than or equal to Smax 
for i in range(n_supply):
    #print(lpSum(dsij_mat[i][j] for j in range(n_demand)) <= Smax[i])
    model += lpSum(dsij_mat[i][j] for j in range(n_demand)) <= Smax[i] , "Supply Constraints " + str(i)

In [14]:
# Demand Constraints
#Row summation must be equal to demand forecast
for j in range(n_demand):
    #print(lpSum(dsij_mat[i][j] for i in range(n_supply)) >= ev_demands[j])
    model += lpSum(dsij_mat[i][j] for i in range(n_supply)) >= ev_demands[j] , "Demand Constraints " + str(j)

model

In [15]:
#Solve the model by minimizing the cost equation 
#by following the supply and demand constraints

model.solve(PULP_CBC_CMD())
status =  LpStatus[model.status]
print(status)

Optimal


In [16]:
#Total cost of customer dissatisfaction
print("Total Cost:", model.objective.value())

Total Cost: 1423560.516345541


In [17]:
# DSij Variable's value
#for v in model.variables():
    #print(v.name,"=", v.value())

In [18]:
#replacing the variable names with the actual value in DSij matrix

dsij_pred=np.zeros((100,4096))
p=0
lst=[]
for v in model.variables():
    k=v.value()
    lst.append(k)
    
for i in range(100):
    for j in range(4096):
        dsij_pred[i][j]=lst[p]
        p+=1
        
table=pd.DataFrame(dsij_pred)
table

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#Checking the Demand constraint (row summation)
sum(table[0])

15.08854

In [20]:
#Comparing with demand value
ev_demands[0]

15.08853972

In [21]:
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Unnamed: 6,Unnamed: 7,smax,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,0,50.163110,19.412014,23,5,3,NaN,NaN,2200,NaN,NaN,NaN,NaN,NaN
1,1,37.336451,58.119225,27,4,7,NaN,NaN,3600,NaN,NaN,NaN,NaN,NaN
2,2,46.709232,57.525650,31,6,14,NaN,NaN,6800,NaN,NaN,NaN,NaN,NaN
3,3,30.528626,55.379835,26,5,5,NaN,NaN,3000,NaN,NaN,NaN,NaN,NaN
4,4,51.521781,35.116755,32,11,6,NaN,NaN,4600,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,NaN,NaN,2200,NaN,NaN,NaN,NaN,NaN
96,96,30.318396,33.388335,32,5,10,NaN,NaN,5000,NaN,NaN,NaN,NaN,NaN
97,97,36.218839,22.235766,32,4,14,NaN,NaN,6400,NaN,NaN,NaN,NaN,NaN
98,98,42.936915,38.122442,28,7,5,NaN,NaN,3400,NaN,NaN,NaN,NaN,NaN


In [22]:
smax19=[]
for i in range(100):
    t=lst[p]
    p+=1
    smax19.append(int(t))
np.array(smax19)

array([ 8200, 10000, 11200,  9400, 10600,  9400,  8200, 11600, 10200,
        7200,  4400,  6600,  7800,  9800, 11200,  8600,  4400,  2000,
       10200, 10000,  8000,  9200,  7000,  8800,  6600, 10000, 11000,
       10000, 10600, 10000,  6600, 12000, 12200, 10400,  6600,  3200,
       11200,  4400,  7000, 10400,  8200,  7800,  9800, 12000,  7000,
       11000, 10600,  7400,  9800, 10600, 10600,  4000, 10200, 13200,
       12200,  5800, 10400,  5800,  6400,  7800,  3600,  3800, 11400,
        6000,  7400, 12400,  5200, 11600,  9600, 11000, 11200, 11600,
        7600,  9800,  3800, 12600,  3200, 10200, 12200, 10600,  8400,
       12600,  7600,  9000,  2200,  8600,  9600,  8600,  6000, 11800,
        8200, 13800,  6400,  9200, 10600,  9000, 11800, 12000,  9800,
        9200])

In [23]:
#Checking the Supply constraint (column summation)
sum(table.iloc[0,:].values)

8199.999978600003

In [24]:
#Comparing with supply value
smax19[0]

8200

In [25]:
supply_set['smax19']=smax19
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Unnamed: 6,Unnamed: 7,smax,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,smax19
0,0,50.163110,19.412014,23,5,3,NaN,NaN,2200,NaN,NaN,NaN,NaN,NaN,8200
1,1,37.336451,58.119225,27,4,7,NaN,NaN,3600,NaN,NaN,NaN,NaN,NaN,10000
2,2,46.709232,57.525650,31,6,14,NaN,NaN,6800,NaN,NaN,NaN,NaN,NaN,11200
3,3,30.528626,55.379835,26,5,5,NaN,NaN,3000,NaN,NaN,NaN,NaN,NaN,9400
4,4,51.521781,35.116755,32,11,6,NaN,NaN,4600,NaN,NaN,NaN,NaN,NaN,10600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,NaN,NaN,2200,NaN,NaN,NaN,NaN,NaN,9000
96,96,30.318396,33.388335,32,5,10,NaN,NaN,5000,NaN,NaN,NaN,NaN,NaN,11800
97,97,36.218839,22.235766,32,4,14,NaN,NaN,6400,NaN,NaN,NaN,NaN,NaN,12000
98,98,42.936915,38.122442,28,7,5,NaN,NaN,3400,NaN,NaN,NaN,NaN,NaN,9800


In [26]:
type(Smax[0])

pulp.pulp.LpVariable

supply_set.to_excel('infra19lb.xlsx')